In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from import_data import read_data

In [3]:
data = read_data('../data', 2014, 2018, 5)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/Users/lthomas/Dropbox/galvanize/Capstone/notebooks/import_data.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data1_cleaned["tweet_time"] = pd.to_datetime(data1_cleaned["tweet_time"])
/Users/lthomas/Dropbox/galvanize/Capstone/notebooks/import_data.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data1_cleane

In [292]:
df = data[ data['year'] == 2015 ]

In [293]:
df.shape

(1644077, 22)

In [32]:
df[["tweet_text", "urls"]].head()

,tweet_text,urls
202461,Arranca el juicio por la corrupción del exgobe...,[http://actualidad.rt.com.ln.is/XvF1n]
8449047,Mattis: ‘We are not winning in Afghanistan rig...,[http://corneey.com/qKRhr5]
7136416,Motorizados solicitan destitución de directora...,[http://dlvr.it/DKJK63]
7496540,Así fue la llorona que montó María Corina en m...,[http://dlvr.it/C7BmDJ]
7588663,Fondos estadounidenses financian terrorismo de...,[http://dlvr.it/8z69mX]


### Grab URLs from Tweets

In [93]:
def grab_urls(tweet):
    return [ word for word in tweet.split() \
                    if word.startswith("http")]

In [98]:
df['real_urls'] = df['tweet_text'].apply(grab_urls)

In [116]:
df['real_urls'].iloc[15]

['https://t.co/aw7j11geLE']

In [110]:
[ word for word in a_tw.split() if not(word.startswith("http")) and not(word.startswith('#'))]

['¡SÉPALO!',
 'Medios',
 'de',
 'comunicación',
 'de',
 'derecha',
 'manipulan',
 'el',
 'término',
 '“terrorismo”',
 'a',
 'conveniencia/',
 '(+VIDEO)']

### Create Bag of Words from Tweets

In [294]:
def get_bag_of_words(tweet):
    return [ word for word in tweet.split() if not(word.startswith("http")) and not(word.startswith('#'))]

In [295]:
df['bag_of_words'] = df['tweet_text'].apply(get_bag_of_words)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [296]:
df[['tweet_text', 'hashtags', 'user_mentions','bag_of_words']].head()

,tweet_text,hashtags,user_mentions,bag_of_words
10002,RT @IFabySalas_: ¿Cómo sería un enfrentamiento...,[],"[IFabySalas_, ForoCandanga]","[RT, @IFabySalas_:, ¿Cómo, sería, un, enfrenta..."
10010,Casi 3 mil mirandinos viven en casas\ninconclu...,[],[],"[Casi, 3, mil, mirandinos, viven, en, casas, i..."
12382,Emir de Catar llega a Venezuela para encuentro...,"[News, Noticias]",[],"[Emir, de, Catar, llega, a, Venezuela, para, e..."
12397,"Senador chileno: ""Chávez sigue vivo en toda Su...","[Noticias, Telesur]",[],"[Senador, chileno:, ""Chávez, sigue, vivo, en, ..."
12400,Un magnate chino es hospitalizado tras colabor...,"[Noticias, News]",[],"[Un, magnate, chino, es, hospitalizado, tras, ..."


In [314]:
a_tw

'¿Realmente participación siete millones de personas en el #Plebiscito inconstitucional de la oposición?… https://t.co/ycs1FyopS3'

In [143]:
import string
punct = set(string.punctuation)
punct.update('¿', '…')

### Create Lexical Diversity Score

In [196]:
def one_tweet_lexical_diversity(tweet):
    def remove_symbols(word, symbol_set):
        return ''.join(char for char in word 
                        if char not in symbol_set)
    
    tokens =  [remove_symbols(word.lower(), punct) for word in tweet.split() 
                            if not(word.startswith("http")) and not(word.startswith('#'))] 
    
    lex_div =  len(set(tokens))/len(tokens)
    return lex_div

In [197]:
def remove_symbols(word, symbol_set):
    return ''.join(char for char in word 
                    if char not in symbol_set)

In [297]:
def grab_user_tweets(tweets_vec):
    all_tokens = []
    for tweet in tweets_vec:
        tokens =  [remove_symbols(word.lower(), punct) for word in tweet.split() 
                            if not(word.startswith("http")) and not(word.startswith('#'))] 
        all_tokens += tokens
    return all_tokens

In [262]:
def user_lexical_diversity(tweets_vec):
    tokens = grab_user_tweets(tweets_vec)  
    if len(tokens) == 0:
        return 0 
    return len(set(tokens))/len(tokens)

In [165]:
df_usergrp = df.groupby('userid')

In [266]:
user_lex_diversity = pd.DataFrame(df_usergrp['tweet_text'].aggregate(diversity)).reset_index()

In [268]:
user_lex_diversity = user_lex_diversity.rename(index=str, columns={"tweet_text": "lexical_diversity_score"})

In [270]:
df = pd.merge(df, user_lex_diversity, how = 'left', on = 'userid')

In [271]:
df.head()

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,user_mentions,reply_count,like_count,retweet_count,year,month,num_tweets,real_urls,bag_of_words,lexical_diversity_score
0,854809163104440320,L3yGWgp5IaIMnGagIFJp2S8L6GTte0XosR+8fDHTdJ4=,L3yGWgp5IaIMnGagIFJp2S8L6GTte0XosR+8fDHTdJ4=,L3yGWgp5IaIMnGagIFJp2S8L6GTte0XosR+8fDHTdJ4=,NaN,Feliz De LaVida #SiguemeYTesigo #AgradecidaCon...,1314,2210,2016-03-16,Arranca el juicio por la corrupción del exgobe...,...,[],0.0,0.0,0.0,2017,4,9285,"[https://t.co/kWuCmbs0GR, https://t.co/t5ckboq...","[Arranca, el, juicio, por, la, corrupción, del...",1.000000
1,874722886480736257,ZTnlUDYlN0WIEoy+KX1PcNvoN3sLsw93BEAm2huShng=,ZTnlUDYlN0WIEoy+KX1PcNvoN3sLsw93BEAm2huShng=,ZTnlUDYlN0WIEoy+KX1PcNvoN3sLsw93BEAm2huShng=,Estados Unidos,"Model, passionate about politics, follow me an...",986,4037,2017-05-31,Mattis: ‘We are not winning in Afghanistan rig...,...,[],0.0,0.0,0.0,2017,6,1451,"[https://t.co/NR3D2tVAKV, https://t.co/fhEKAGt...","[Mattis:, ‘We, are, not, winning, in, Afghanis...",1.000000
2,689488058916319234,+uoYQvK+gZIHig5Aim5NboS0CXcxTG0ZdAqp844U=,+uoYQvK+gZIHig5Aim5NboS0CXcxTG0ZdAqp844U=,+uoYQvK+gZIHig5Aim5NboS0CXcxTG0ZdAqp844U=,NaN,"ANTI: Chavista, Castrista, Madurista y todo lo...",3413,2208,2013-05-27,Motorizados solicitan destitución de directora...,...,[],0.0,0.0,0.0,2016,1,154746,[https://t.co/r4NEPUa1Xb],"[Motorizados, solicitan, destitución, de, dire...",0.775510
3,642432189464838144,QxktQFbfj2TAIHamCgPJ17mrkhHMu6XlCM7dQZXJgmM=,QxktQFbfj2TAIHamCgPJ17mrkhHMu6XlCM7dQZXJgmM=,QxktQFbfj2TAIHamCgPJ17mrkhHMu6XlCM7dQZXJgmM=,Salvemos el Planeta!!,Divertida y coqueta son las palabras que me de...,747,272,2015-05-10,Así fue la llorona que montó María Corina en m...,...,[],0.0,0.0,0.0,2015,9,112342,"[http://t.co/LPpTya0H52, http://t.co/CdJWXAOLVx]","[Así, fue, la, llorona, que, montó, María, Cor...",0.873239
4,577097570717069312,wHhZVjZKB0OZb27lHj3J5i7Udmr9ROdU5+oqKH3mZ8s=,wHhZVjZKB0OZb27lHj3J5i7Udmr9ROdU5+oqKH3mZ8s=,wHhZVjZKB0OZb27lHj3J5i7Udmr9ROdU5+oqKH3mZ8s=,#Vzla,"Mujer de hogar, con valores agradecida de la f...",3705,1470,2014-03-26,Fondos estadounidenses financian terrorismo de...,...,[],0.0,0.0,0.0,2015,3,92895,[http://t.co/ksZomDtgqU],"[Fondos, estadounidenses, financian, terrorism...",0.925926


### Grab Content from URLs

In [193]:
import requests
from bs4 import BeautifulSoup

In [279]:
def grab_url_content(url):
    url = url
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    title = soup.find('title').decode()
    title = title[7:-8]   
    return title

In [276]:
df['real_urls'].head(25)

0     [https://t.co/kWuCmbs0GR, https://t.co/t5ckboq...
1     [https://t.co/NR3D2tVAKV, https://t.co/fhEKAGt...
2                             [https://t.co/r4NEPUa1Xb]
3      [http://t.co/LPpTya0H52, http://t.co/CdJWXAOLVx]
4                              [http://t.co/ksZomDtgqU]
5                             [https://t.co/te5FqhSgCZ]
6     [https://t.co/eTM00ubNoO, https://t.co/jubW8ds...
7                                                    []
8      [http://t.co/UyQj8O8mav, http://t.co/jQMFOVgyys]
9     [https://t.co/CFQpfFdnHW, https://t.co/pan3NmB...
10    [https://t.co/FfjLTP6Wo0, https://t.co/9y0GR2d...
11                                [http://t.co/90uxmj…]
12    [https://t.co/Mzvmd6dFKa, https://t.co/Dbe0VRo...
13                                                   []
14    [https://t.co/rfzbVnoDdh, https://t.co/zj6A0Ca...
15                            [https://t.co/aw7j11geLE]
16                                                   []
17    [https://t.co/qgU3mQR40U, https://t.co/YY2

In [285]:
def grab_all_urls(urls):
    content = []
    for url in urls:
        if len(url) == 0:
            return ""
        result = grab_url_content (url)
        if result == 'Twitter / Account Suspended':
            result = ''
        content.append( result )
    return content

In [289]:
grab_all_urls(df['real_urls'][2])

['FeedsPortal.com']

In [291]:
df['real_urls'][:3].apply(grab_all_urls)

['https://t.co/NR3D2tVAKV', 'https://t.co/fhEKAGtUqh']

### Create User/tweet Documents

In [298]:
users_df_grp = df.groupby('userid')

In [302]:
user_doc = users_df_grp['tweet_text'].apply(grab_user_tweets)

In [305]:
user_doc = pd.DataFrame(user_doc).reset_index().rename(index=str, columns={"tweet_text": "tweet_document"})

In [307]:
user_doc.head()

,userid,tweet_document
0,+dQ+QlvsYdF8Lald5LNFZRmGAJVayY9jpoeDm4mvMko=,"[deaaaadd, si, hoy, no, te, fue, bien, tranqui..."
1,+kKXxTxO9bdLP3qG1Pg8duf8g3LJk2ywVN6+Vr551M=,"[lacelis, deberías, cambiarte, pero, de, canal..."
2,+mGUsHi4yPJaHRZAJDORQlejcMd2XylmlIg6HAl5e4k=,"[kesaraduque572, holaa, guapa, rt, oficinaship..."
3,+qqdfdDCLuIvMb5qa5h1tu0zpRaI4Kc35u6SCBBGIg8=,"[janiot, ayotzinapa, gaza, ferguson, y, donets..."
4,+tWUswUHZ6Wyjd9Po8905nfjk800K5pAwr5EvxHIC8=,"[rt, m0nedavenezuela, medalla, conmemorativa, ..."


In [309]:
from langdetect import detect

### Closing the Loop

In [326]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [328]:
# Create a dictionary from tweets containing the number of times a word appears in the training set.

In [337]:
dictionary = gensim.corpora.Dictionary(user_doc['tweet_document'])

In [338]:
len(dictionary)

242494

In [339]:
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [340]:
len(dictionary)

62475

In [341]:
# Each Doc, Create Dictionary BoW Counts

In [342]:
bow_corpus = [dictionary.doc2bow(doc) for doc in user_doc['tweet_document']]

In [343]:
len(bow_corpus)

307

In [344]:
# TF-IDF

In [346]:
from gensim import corpora, models
from pprint import pprint

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]



In [347]:
# LDA on BoW

In [348]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [351]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.003*"islámico" + 0.003*"siria" + 0.002*"ruso" + 0.002*"méxico" + 0.002*"españa" + 0.002*"cuba" + 0.002*"argentina" + 0.002*"putin" + 0.001*"muertos" + 0.001*"fanb"
Topic: 1 
Words: 0.004*"forocandanga" + 0.002*"obama" + 0.002*"cuba" + 0.001*"islámico" + 0.001*"onu" + 0.001*"méxico" + 0.001*"anat5" + 0.001*"muertos" + 0.001*"putin" + 0.001*"frontera"
Topic: 2 
Words: 0.008*"forocandanga" + 0.005*"conelmazodando" + 0.003*"siria" + 0.003*"méxico" + 0.003*"anat5" + 0.003*"yudithh99" + 0.002*"islámico" + 0.002*"cuba" + 0.002*"argentina" + 0.002*"muertos"
Topic: 3 
Words: 0.007*"ojo" + 0.004*"islámico" + 0.003*"siria" + 0.003*"putin" + 0.002*"ruso" + 0.002*"muertos" + 0.002*"vladimirpadrino" + 0.002*"ucrania" + 0.002*"méxico" + 0.002*"onu"
Topic: 4 
Words: 0.008*"vladimirpadrino" + 0.007*"fanb" + 0.004*"ceofanb" + 0.004*"ejercitofanb" + 0.003*"gnboficial" + 0.003*"armadafanb" + 0.003*"gnb" + 0.002*"redimain" + 0.002*"gestión" + 0.002*"frontera"
Topic: 5 
Words: 0.002*"siri

In [349]:
# LDA on TF-IDF

In [350]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [352]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.000*"golpeados" + 0.000*"elnacionalweb" + 0.000*"animo" + 0.000*"guardianesl" + 0.000*"hectorodriguez" + 0.000*"genero" + 0.000*"msc" + 0.000*"minmujer" + 0.000*"ovacionve" + 0.000*"ayúdame"
Topic: 1 
Words: 0.001*"vladimirpadrino" + 0.000*"esequibovzla" + 0.000*"yudithh99" + 0.000*"ceofanb" + 0.000*"fanb" + 0.000*"redimain" + 0.000*"ejercitofanb" + 0.000*"armadafanb" + 0.000*"gnboficial" + 0.000*"encuentro"
Topic: 2 
Words: 0.000*"ttejuanescalona" + 0.000*"cocodrilosdeccs" + 0.000*"mincioficial" + 0.000*"aki" + 0.000*"angelicarojas24" + 0.000*"liviarevenga" + 0.000*"laenamoran" + 0.000*"logicademujeres" + 0.000*"milicia" + 0.000*"posted"
Topic: 3 
Words: 0.000*"móvildisponible" + 0.000*"store" + 0.000*"vladimirpadrino" + 0.000*"play" + 0.000*"mire" + 0.000*"app" + 0.000*"ceofanb" + 0.000*"google" + 0.000*"lt3" + 0.000*"fil0s0fia"
Topic: 4 
Words: 0.000*"hanthonycoello" + 0.000*"eterno" + 0.000*"ojo" + 0.000*"endycontreras" + 0.000*"sorayaachkar" + 0.000*"hola" + 0.0

In [353]:
# Attempt to Visualize

In [355]:
import pyLDAvis.gensim

In [356]:
tw_data =  pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
pyLDAvis.display(tw_data)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
